## YaSpeller
**Проверка орфографии**

In [9]:
from bs4 import BeautifulSoup
import urllib

In [152]:
def check_spell(word):
    path = 'https://speller.yandex.net/services/spellservice/checkText?text=' + urllib.parse.quote(word)
    response = urllib.request.urlopen(path)
    correct = BeautifulSoup(response.read()).find_all('s')[0].get_text()
    return correct

In [154]:
check_spell("кампьютер")

'компьютер'

In [155]:
check_spell("ghbdtn")

'привет'

## YaDictionary
**Поиск синонимов**

In [91]:
import sys 
import config
sys.path.append('portYaDict.py')
from portYaDict import YandexDictionary

In [92]:
def search_syn(word):
    
    # Получаем ответ API на запрос по слову
    dictionary = YandexDictionary(config.API_DICT_KEY)
    output = dictionary.lookup(word, 'ru', 'ru')
    
    # Фильтруем (очищаем) данные
    russian_letters = set("ёйцукенгшщзхъфывапролджэячсмитьбю ")
    russian_text = ""
    for letter in output:
        if letter in russian_letters:
            russian_text += letter    
        else:
            russian_text += ","
    
    # Массив синонимов, возвращаем первые 5
    synonymus = []
    synonymus = list(filter(lambda x: len(x.split(" ")) == 1 and len(x) > 0, list(russian_text.split(','))))
    
    return synonymus[0:5]

In [93]:
search_syn("город")

['город', 'сити', 'мегаполис', 'г', 'град']

## Compare texts

### With difflib
**Построение векторов**

In [35]:
import difflib, time

In [52]:
def similarity_diff(text1, text2):
    start_time = time.time()
    
    normalized1 = text1.lower()
    normalized2 = text2.lower()
    matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
    
    work_time = round(time.time() - start_time)
    print("Work time: {0}".format(work_time))
    return matcher.ratio()

In [72]:
sim_diff = similarity_diff("Мясо гигантских тараканов станет вкусной и недорогой альтернативой говядине",
                           "Гигантское мясо тараканов станет говядине недорогой и вкусной альтернативой")
print(sim_diff)

Work time: 0
0.6933333333333334


### With sklearn
**Тоже построение векторов (только лучше) + косинусное расстояние**

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [78]:
def similarity_sklearn(text1, text2):
    start_time = time.time()
    
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform([text1, text2])

    work_time = round(time.time() - start_time)
    print("Work time: {0}".format(work_time))
    return(tfidf * tfidf.T).A[0][1]

In [82]:
sim_sklearn = similarity_sklearn("Мясо гигантских тараканов станет вкусной и недорогой альтернативой говядине",
                                 "Гигантское мясо тараканов станет говядине недорогой и вкусной альтернативой")
print(sim_sklearn)

Work time: 0
0.7799154245579976


## Text clustering
**Сравнение векторов**

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = ["Хлеб — хлебобулочное изделие без начинки",
             "Для приготовления хлеба употребляют пшеничную и ржаную муку, реже — кукурузную, ячменную и другие",
             "Хлеб можно есть отдельно, однако нередко его едят со сливочным, арахисовым или подсолнечным маслом",
             "Хлеб — один из старейших приготавливаемых продуктов, появившийся ещё в неолите",
             "Вода является хорошим сильнополярным растворителем.",
             "Исключительно важна роль воды в глобальном кругообороте вещества и энергии",
             "Это солёная вода, непригодная для сельского хозяйства и питья",
             "Пресная же вода находится в основном в ледниках"]

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["Люблю пить воду"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["Ем хлеб с утра и вечером с маслом"])
prediction = model.predict(Y)
print(prediction)
 

Top terms per cluster:
Cluster 0:
 хлеб
 изделие
 без
 хлебобулочное
 начинки
 старейших
 продуктов
 один
 неолите
 приготавливаемых
Cluster 1:
 вода
 для
 является
 сильнополярным
 хорошим
 растворителем
 основном
 пресная
 же
 находится


Prediction
[1]
[0]


In [70]:
def getReliabilityByInn(inn):
    contragent = "https://sbis.ru/contragents/"
    
    f = urllib.request.urlopen(contragent+str(inn))
    
    contents = f.read()
    
    soup = BeautifulSoup(contents, 'lxml') 
    
    reliability = soup.find("div", {"class": "cCard__Reliability"})
    
    plus = reliability.find("div", {"class": "analytics-ReliabilitySbisRu__smallTextGreen analytics-ReliabilitySbisRu__left"})
    
    minus = reliability.find("div", {"class": "analytics-ReliabilitySbisRu__smallTextRed analytics-ReliabilitySbisRu__left"})

    plus_num = str(plus)[str(plus).rfind('+')+2:str(plus).rfind('<')]
    
    minus_num = str(minus)[str(minus).rfind('-')+2:str(minus).rfind('<')]
        
    f.close()  
    
    # print('+' + plus_num)
    
    # print('-' + minus_num)
    
    return float(plus_num) - float(minus_num)

## Инн + рейтинг 

In [28]:
import pandas as pd 
import numpy as np 

In [22]:
data = pd.read_excel('Main.xlsx')

In [40]:
data_dict = dict(zip(list(data.iloc[:,0]), np.array(data.iloc[:,[1,2,4,5]])))

In [61]:
df = pd.DataFrame(data)
df

,Способ закупки,Дата заключения,Заказчик,Регион заказчика,ИНН заказчика,КПП заказчика,Реестровый номер,Номер,Предмет,Наименование позиции,Количество,Цена за единицу,Идентификатор СТЕ,Поставщик,Регион поставщика,ИНН поставщика,КПП поставщика
0,Другое (импорт),2019-07-26 00:00:00.000,Государственное бюджетное образовательное учре...,г Москва,7715107749,771501001.0,2771510774919000064,962_2865,Поставка форменного обмундирования для кадет (...,NaN,34.0,937.03324,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САВИ...",г Санкт-Петербург,7825058290,784001001.0
1,Другое (импорт),2019-07-26 00:00:00.000,Государственное бюджетное образовательное учре...,г Москва,7715107749,771501001.0,2771510774919000064,962_2865,Поставка форменного обмундирования для кадет (...,NaN,18.0,1087.96222,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САВИ...",г Санкт-Петербург,7825058290,784001001.0
2,Другое (импорт),2019-07-26 00:00:00.000,Государственное бюджетное образовательное учре...,г Москва,7715107749,771501001.0,2771510774919000064,962_2865,Поставка форменного обмундирования для кадет (...,NaN,20.0,267.62850,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САВИ...",г Санкт-Петербург,7825058290,784001001.0
3,Другое (импорт),2019-07-26 00:00:00.000,Государственное бюджетное образовательное учре...,г Москва,7715107749,771501001.0,2771510774919000064,962_2865,Поставка форменного обмундирования для кадет (...,NaN,34.0,792.72853,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САВИ...",г Санкт-Петербург,7825058290,784001001.0
4,Другое (импорт),2019-07-26 00:00:00.000,Государственное бюджетное образовательное учре...,г Москва,7715107749,771501001.0,2771510774919000064,962_2865,Поставка форменного обмундирования для кадет (...,NaN,54.0,248.41333,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САВИ...",г Санкт-Петербург,7825058290,784001001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942961,Другое (импорт),2019-12-05 00:00:00.000,"ГАУЗ ""СП № 62 ДЗМ""",г Москва,7724245826,772401001.0,57724245826190002094080,31908513009,Оказание услуг по проведению периодических мед...,Оказание услуг по проведению медицинского осмо...,1.0,1103.13000,NaN,"ООО ""МАГНЕТОМ-М""",г Москва,7726731814,772601001.0
942962,Другое (импорт),2019-12-05 00:00:00.000,"ГАУЗ ""СП № 62 ДЗМ""",г Москва,7724245826,772401001.0,57724245826190002094080,31908513009,Оказание услуг по проведению периодических мед...,Оказание услуг по проведению медицинского осмо...,1.0,1043.97000,NaN,"ООО ""МАГНЕТОМ-М""",г Москва,7726731814,772601001.0
942963,Другое (импорт),2019-12-05 00:00:00.000,"ГАУЗ ""СП № 62 ДЗМ""",г Москва,7724245826,772401001.0,57724245826190002094080,31908513009,Оказание услуг по проведению периодических мед...,Оказание услуг по проведению медицинского осмо...,1.0,879.92000,NaN,"ООО ""МАГНЕТОМ-М""",г Москва,7726731814,772601001.0
942964,Другое (импорт),2019-12-05 00:00:00.000,"ГАУЗ ""СП № 62 ДЗМ""",г Москва,7724245826,772401001.0,57724245826190002094080,31908513009,Оказание услуг по проведению периодических мед...,Оказание услуг по проведению медицинского осмо...,58.0,1208.02000,NaN,"ООО ""МАГНЕТОМ-М""",г Москва,7726731814,772601001.0


In [115]:
def get_rate(df):
    rates = []
    df = df.values
    for string in df:
        try:
            rate = getReliabilityByInn(string[4])
        except Exception:
            rate = 0
        rates.append(rate)
        print("Inn: {0} -- Rate:{1}".format(string[4], rate))
        print(rates)
    return rates    

In [121]:
rates = get_rate(df[931990:932000])

Inn: 7728381587 -- Rate:0
Inn: 7724857843 -- Rate:0
Inn: 7731435497 -- Rate:0
Inn: 7724941534 -- Rate:0
Inn: 7724941534 -- Rate:0
Inn: 7724941534 -- Rate:0
Inn: 7718193658 -- Rate:0
Inn: 7718193658 -- Rate:0
Inn: 7701984274 -- Rate:99.8
Inn: 7701984274 -- Rate:99.8


In [125]:
d['Рейтинг'] = rates 

C:\Users\Fedor\Anaconda3\envs\tenderH\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [124]:
d

,Способ закупки,Дата заключения,Заказчик,Регион заказчика,ИНН заказчика,КПП заказчика,Реестровый номер,Номер,Предмет,Наименование позиции,Количество,Цена за единицу,Идентификатор СТЕ,Поставщик,Регион поставщика,ИНН поставщика,КПП поставщика,Rate
931990,КС,2019-12-11 00:00:00.000,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОС...,г Москва,7728381587,772801001.0,19-3096173,173-МА-12-19,Поставка новогодней атрибутики,Предметы внутреннего интерьера,2.000,248.75,NaN,ООО «МОРОЗКО-М»,г Москва,7726732600,772601001.0,0.0
931991,КС,2019-12-11 00:00:00.000,Государственное бюджетное общеобразовательное ...,г Москва,7724857843,772401001.0,19-3100090,263/1466-2019,Поставка товаров для уроков технологии в ГБОУ ...,Ткани; белый/бежевый; 150 см; пирожное; 10 мет...,20.000,238.80,NaN,ИП Желнакова Елена Николаевна,обл Воронежская,366405162737,NaN,0.0
931992,КС,2019-12-03 00:00:00.000,Государственное бюджетное общеобразовательное ...,г Москва,7731435497,773101001.0,19-3058844,М187/97-19,Поставка товаров для нужд ГБОУ Школа № 97,Унитазы; белый,3.000,1962.86,NaN,"ООО ""СПЕЦСТРОЙ""",г Москва,7722328900,773301001.0,0.0
931993,Другое (импорт),2019-09-02 00:00:00.000,Государственное бюджетное учреждение города Мо...,г Москва,7724941534,772401001.0,2772494153418999808,0173200001419000962-0661302-1,Выполнение работ по ремонту асфальтобетонного ...,Замена бортового камня дорожного в рамках благ...,340.000,1345.27,NaN,"ООО ""ДорСтройРесурс""",г Москва,7728828000,772701001.0,0.0
931994,Другое (импорт),2019-09-02 00:00:00.000,Государственное бюджетное учреждение города Мо...,г Москва,7724941534,772401001.0,2772494153418999808,0173200001419000962-0661302-1,Выполнение работ по ремонту асфальтобетонного ...,Замена бортового камня дорожного в рамках благ...,140.000,1356.55,NaN,"ООО ""ДорСтройРесурс""",г Москва,7728828000,772701001.0,0.0
931995,Другое (импорт),2019-09-02 00:00:00.000,Государственное бюджетное учреждение города Мо...,г Москва,7724941534,772401001.0,2772494153418999808,0173200001419000962-0661302-1,Выполнение работ по ремонту асфальтобетонного ...,Замена бортового камня дорожного в рамках благ...,655.000,1348.42,NaN,"ООО ""ДорСтройРесурс""",г Москва,7728828000,772701001.0,0.0
931996,Другое (импорт),2019-12-11 00:00:00.000,Государственное бюджетное общеобразовательное ...,г Москва,7718193658,771801001.0,2771819365818999808,0873500000819004899-1360,"Поставка игрушек, игрового оборудования и това...",Детский костюм «Почтальон» в рамках проекта «Т...,2.000,365.11,NaN,"Общество с ограниченной ответственностью ""РАЗ...",обл Ивановская,3702115431,370201001.0,0.0
931997,Другое (импорт),2019-12-11 00:00:00.000,Государственное бюджетное общеобразовательное ...,г Москва,7718193658,771801001.0,2771819365818999808,0873500000819004899-1360,"Поставка игрушек, игрового оборудования и това...",Детский костюм «Полицейский» в рамках проекта ...,2.000,296.40,NaN,"Общество с ограниченной ответственностью ""РАЗ...",обл Ивановская,3702115431,370201001.0,0.0
931998,Другое (импорт),2019-12-09 00:00:00.000,"АО ""МОСВОДОКАНАЛ""",г Москва,7701984274,770101001.0,57701984274190000193536,2360-19,"Выполнение работ по объекту: ""Подключение к це...","Реконструкция трубопроводов водопровода, км",0.117,38752665.21,NaN,"ООО ""ТАНАЙС""",обл Московская,5027267152,502701001.0,99.8
931999,Другое (импорт),2019-12-10 00:00:00.000,"АО ""МОСВОДОКАНАЛ""",г Москва,7701984274,770101001.0,57701984274190000193536,ПД-181-2019,"Выполнение работ по объекту: ""Подключение к ...","Реконструкция трубопроводов водопровода, км",0.106,40751372.83,NaN,"ООО ""ВЕКТОР-Ф""",обл Московская,7726723637,502701001.0,99.8
